In [1]:
import os
import openai
import tiktoken
# from dotenv import load_dotenv
import time
import re
# from google.colab import drive
# drive.mount('/content/gdrive')
# path = '/content/gdrive/My Drive/585data/'

In [2]:
# path = '../data/'    # change the path as needed
path = '../' #'/content/gdrive/My Drive/585data/'

def read_data(file):
    with open (path+file) as t:
        data = t.readlines()
    return data

test_set = read_data('test_data.txt')[:60]

# type(train_set)
print(test_set[:2], '\n', len(test_set))

['{"groundTruth": ["现身说法"], "candidates": [["旷日持久", "公正廉洁", "苦口婆心", "现身说法", "白日做梦", "深入浅出", "肺腑之言"]], "content": "只要路过的旅客稍有迟疑，或者对他们的宣传单多看几眼，基本上这个旅客就别想轻松脱身了，记者就在9月3日接站时目睹了这样一幕：一个学生接过招生人员递来的宣传单，只是问了一下“你们学校有没有分数要求？”两个招生人员就“白话”开了，一个表示分数都好说，只要有好学的精神；另一个则#idiom#，大讲自己选择的专业现在收获颇丰；最后在招生人员“我们学校毕业后可以完全解决就业”的忽悠下，这个学生旅客被他们拉上了到校参观的班车。", "realCount": 1}\n', '{"groundTruth": ["神来之笔", "赞不绝口"], "candidates": [["画龙点睛", "悔过自新", "拍案叫绝", "鬼斧神工", "神来之笔", "颠倒黑白", "中流砥柱"], ["敬谢不敏", "拍案叫绝", "心悦诚服", "叹为观止", "赞不绝口", "口口声声", "扬眉吐气"]], "content": "亨利的这个#idiom#被法国媒体形容为“空中舞蹈”，亨利自己对球队表现也很满意，“上半场开局一般，但很快觉醒，下半场的进攻让人看到真正的法国，尤其是我们的速度让对方有了麻烦。”而和亨利搭档的本泽马对老大哥#idiom#，“和他在一起配合很容易，他给了我很多信心。”", "realCount": 2}\n'] 
 60


In [3]:
# load_dotenv()

api_path = path + 'openai_gpt_api_key.txt'

### read api key from file
with open(api_path, "r") as file:
    API_KEY = file.read().strip()

openai.api_key = API_KEY
enc = tiktoken.get_encoding("cl100k_base")

messages = []

def ask_gpt(input_text, print_results = True, delay: float = 1):
    """
    API call to gpt-3.5-turbo. Note: each time the function is called, the API call will keep previous calls to the model as context until the kernel is restarted.
    If you would like to reset the model's conversational context, restart kernel and run the function again.

    Parameters:
    ------
    `input`: str
        user input that is passed to model. Used in the same way information is passed to the model in the OpenAI web interface for ChatGPT
    `print_results`: bool = True
        if model's returned text exceeds a certain number of characters, it includes new line marking "\n". if print_result == True,
        function splits printout by "\n" and print results out line by line, otherwise results are only returned as a list object,
        with each list item as a line of the printout
    `delay`: float
        amount by which to delay sequential API calls (calls in sequence that are too frequent can lead to a ban in certain instances - this avoids that)
    
    Returns:
    ------
    `result_list`: list
        only returned if `print_result` == False. Return is a list object with returned text from the model split by "\n".
        If returned text is short enough and no text is present, list will only be length of 1
    """
    messages = []
    
    time.sleep(delay)
    
    # global messages
    messages.append({"role": "user", "content": input_text})
    chat_test = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        temperature = 0.1,
        messages = messages)
    
    result_list = chat_test.choices[0].message.content#.split("\n") # if not split, the returned text is more than one line, the printout has "\n" in it

    # if print_results == True:
    #     for line in result_list:
    #         print (line)
    # else:
    return result_list


In [4]:
def prompt(data):
    '''data shall be the output of `read_data`'''
    text_input = []
    gold = []
    for i in range(len(data)):
        data[i] = eval(data[i])
        input_text = data[i]['content']
        candidates = data[i]['candidates']
        ground_truth = set(data[i]['groundTruth'])
        gold.append(ground_truth)

        candidate_str = ''
        for candidate in candidates:
            candidate_str += '('+'|'.join(candidate)+')'
        
        # preprocess_idx = -1
        # def replace(match):
        #     nonlocal preprocess_idx
        #     preprocess_idx += 1
        #     return 'extra{}'.format(preprocess_idx)
        # input_text = re.sub(r'#idiom#', replace, input_text)

        instruction = f'''示例：(意气风发|街谈巷议|人才辈出|一脉相传|后继有人|发扬光大|腥风血雨)(平易近人|落落大方|八仙过海|彬彬有礼|史无前例|盛气凌人|好自为之)(不拘小节|风流潇洒|无病呻吟|言谈举止|壮志凌云|关门闭户|温文尔雅)
历史上的蒋南翔是著名的“一二九”学生救亡运动的领导人之一，他在清华校长之位14年期间，不但很好的继承了清华建校之初的优秀传统与理念，而且更加的#idiom#，他把清华的教师队伍扩大了将近5倍，将清华本科人数破万，为新中国培养了大量的有用人才。在《天行健》中饰演蒋南翔的刘威是观众所熟悉的著名实力派演员，早在1987年刘威就在《关东大侠》中饰演豪爽仗义的关云天一角而获得了金鸡奖最佳男主角的提名，后来更是因在《唐明皇》中精湛的表演而一举夺得金鹰奖最佳男演员奖。此次《天行健》选定刘威来出演正是看中了他#idiom#的表演方式和对人物深入内心的刻画。至此，《天行健》中涉及的三位清华校长的人选都已经曝光，#idiom#的第一任校长赵文?、稳重坚毅的第二任校长孙逊、亲切务实的第三任校长刘威，再加上梁思成、林徽因、朱自清、闻一多等一批“大师”的加盟，相信作为清华百年校庆重点项目之一的《天行健》一定会带领观众重温那段不能抹去的历史。 
示例的回答为：发扬光大,平易近人,温文尔雅
请按照示例从下列括号中分别选择合适的成语填入"#idiom#"处：{candidate_str}'''
        
        text_input.append(instruction+'\n'+input_text)

    return text_input, gold

In [5]:
test_input, gold = prompt(test_set)

In [6]:
test_input[1]

'示例：(意气风发|街谈巷议|人才辈出|一脉相传|后继有人|发扬光大|腥风血雨)(平易近人|落落大方|八仙过海|彬彬有礼|史无前例|盛气凌人|好自为之)(不拘小节|风流潇洒|无病呻吟|言谈举止|壮志凌云|关门闭户|温文尔雅)\n历史上的蒋南翔是著名的“一二九”学生救亡运动的领导人之一，他在清华校长之位14年期间，不但很好的继承了清华建校之初的优秀传统与理念，而且更加的#idiom#，他把清华的教师队伍扩大了将近5倍，将清华本科人数破万，为新中国培养了大量的有用人才。在《天行健》中饰演蒋南翔的刘威是观众所熟悉的著名实力派演员，早在1987年刘威就在《关东大侠》中饰演豪爽仗义的关云天一角而获得了金鸡奖最佳男主角的提名，后来更是因在《唐明皇》中精湛的表演而一举夺得金鹰奖最佳男演员奖。此次《天行健》选定刘威来出演正是看中了他#idiom#的表演方式和对人物深入内心的刻画。至此，《天行健》中涉及的三位清华校长的人选都已经曝光，#idiom#的第一任校长赵文?、稳重坚毅的第二任校长孙逊、亲切务实的第三任校长刘威，再加上梁思成、林徽因、朱自清、闻一多等一批“大师”的加盟，相信作为清华百年校庆重点项目之一的《天行健》一定会带领观众重温那段不能抹去的历史。 \n示例的回答为：发扬光大,平易近人,温文尔雅\n请按照示例从下列括号中分别选择合适的成语填入"#idiom#"处：(画龙点睛|悔过自新|拍案叫绝|鬼斧神工|神来之笔|颠倒黑白|中流砥柱)(敬谢不敏|拍案叫绝|心悦诚服|叹为观止|赞不绝口|口口声声|扬眉吐气)\n亨利的这个#idiom#被法国媒体形容为“空中舞蹈”，亨利自己对球队表现也很满意，“上半场开局一般，但很快觉醒，下半场的进攻让人看到真正的法国，尤其是我们的速度让对方有了麻烦。”而和亨利搭档的本泽马对老大哥#idiom#，“和他在一起配合很容易，他给了我很多信心。”'

In [7]:
gold[:2]

[{'现身说法'}, {'神来之笔', '赞不绝口'}]

In [8]:
ask_gpt(test_input[1], delay = 0.01)

'神来之笔,拍案叫绝'

In [9]:
sys = []
for q in test_input:
    a = ask_gpt(q, delay = 18)
    a = a.strip(' 。，答案：').split(',')
    sys.append(set(a))

In [12]:
with open('chatgpt_ans.txt', 'w') as a:
    for s in sys:
        a.write(str(s)+'\n')    

In [13]:
def f1_score(sys, gold):
    tp = 0
    total = 0
    pos = 0
    for s, g in zip(sys, gold):
        total += len(g)
        pos += len(s)
        tp += len(g & s)
    precision = tp / pos if pos != 0 else 0
    recall = tp / total if total != 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    return precision, recall, f1, tp

def accuracy(sys, gold, tp):
    total = 0
    for s, g in zip(sys, gold):
        total += len(g)
    return tp / total

In [14]:
p, r, f1, tp = f1_score(sys, gold)
print('ChatGPT: ')
print(f"Accuracy for test set is {accuracy(sys, gold, tp)}")
print(f"F1 score for test set is {f1}")

ChatGPT: 
Accuracy for test set is 0.24615384615384617
F1 score for test set is 0.2424242424242424


In [16]:
p = 0
for s, g in zip(sys, gold):
    if s != g:
        print(s, g)
        p += 1

{'深入浅出'} {'现身说法'}
{'神来之笔', '拍案叫绝'} {'神来之笔', '赞不绝口'}
{'劳燕分飞'} {'难分难舍'}
{'望而生畏'} {'不寒而栗'}
{'插翅难飞'} {'凶多吉少'}
{'恬不知耻'} {'堂而皇之'}
{'气宇轩昂，满面春风'} {'精神抖擞'}
{'墨守成规，折戟沉沙，怀才不遇'} {'闭门造车'}
{'恰如其分'} {'锦上添花'}
{'光明磊落'} {'快人快语'}
{'繁文缛礼', '兵戎相见'} {'一触即发'}
{'#泰然自若#，#左顾右盼#的叶尔莫拉耶夫集团军长指挥着部队进攻，#分工合作#的各个部队密切配合，最终取得了胜利。而德军则#惊惶失措#，#猝不及防#，被我军打得节节败退'} {'全神贯注', '惊惶失措'}
{'一般无二'} {'本来面目'}
{'怒火中烧', '自力更生'} {'自力更生', '气势汹汹'}
{'分门别类，按图索骥'} {'分门别类'}
{'骑虎难下，居心叵测'} {'狗急跳墙'}
{'东窗事发'} {'冒名顶替'}
{'返朴归真', '众星攒月', '茅塞顿开'} {'众星攒月'}
{'满面春风', '视若无睹', '坦然自若'} {'坦然自若'}
{'口口相传，津津乐道，约定俗成'} {'口口相传'}
{'富可敌国'} {'白手起家'}
{'然而，这些对理科生的刻板印象在近年来已经被打破，越来越多的理科生开始#idiom#地走向大众视野，他们不仅在学术领域有所建树，还在文化、艺术等领域有所涉猎和贡献。他们不再#idiom#地局限于自己的领域，而是积极地参与到社会生活中，成为了社会发展的重要力量'} {'周而复始'}
{'岌岌可危，以攻为守，牢不可破'} {'土崩瓦解'}
{'以退为进'} {'明哲保身'}
{'任劳任怨，笑容可掬，苦心孤诣'} {'笑容可掬'}
{'老成持重'} {'满面春风'}
{'但是这个换人决定非但没能解决场上的问题，还更加暴露了利物浦目前的艰难处境：中场缺少一个冷静的大脑———阿奎拉尼上场后，出球屡屡慢半拍，被对手铲得人仰马翻。此前杰拉德打后腰位置虽然遏制了他的进攻能力，但在防守上还是有保障的。意大利中场替补上来后，让搭档卢卡斯痛快淋漓。他对于比赛节奏的掌控，并没有在过去两场联赛中改进，一遇到打法粗野的英冠球队愈发示弱'} {'疲於奔命', '人仰马翻'